In [2]:
import numpy as np
import math
import os
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
import pathlib
import sys
import natsort
from skimage.feature import peak_local_max
from skimage import util, measure
from skimage.filters import threshold_otsu, gaussian
from skimage.segmentation import watershed
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import closing,  disk, dilation, erosion, ball
from scipy.ndimage import distance_transform_edt
from scipy import ndimage as ndi
from skimage.transform import rescale

import napari
from tqdm import tqdm
from tifffile import imread
import tifffile

import warnings
warnings.filterwarnings("ignore")



In [ ]:
#IGNORE THIS - SOMETIMES THE DECONVOLUTION RETURNS IMAGES WITH PIXEL VALUES ALL OVER THE PLACE, SO THIS CONVERTS EVERY IMAGE TO HAVE PIXEL VALUES BETWEEN 0 AND 255
def convert(img, target_type_min, target_type_max, target_type):
    """
    Converts an image to a specified data type while scaling its intensity values.

    This function rescales the intensity values of an image from its original range 
    to a new target range specified by `target_type_min` and `target_type_max`, and 
    then converts it to the desired data type.

    This step is required as deconvolved images are not always scaled 0->255! 

    Parameters:
    -----------
    img : numpy.ndarray
        The input image array to be converted.
    target_type_min : int or float
        The minimum value of the target intensity range.
    target_type_max : int or float
        The maximum value of the target intensity range.
    target_type : numpy.dtype
        The desired data type of the output image (e.g., np.uint8, np.float32).

    Returns:
    --------
    new_img : numpy.ndarray
        The rescaled image with values mapped to the new intensity range and converted 
        to the specified data type.

    Notes:
    ------
    - This function performs a linear transformation to scale pixel values.
    - It ensures that the output values are properly mapped between `target_type_min` and 
      `target_type_max`.
    """
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img

In [ ]:
#IGNORE THIS - EXTRACTS TIF SIZE FROM IMAGE METADATA
def pixel_size(tif_path):
    """
    Extracts the pixel size (spacing) in microns from a TIFF image.

    This function reads TIFF metadata to determine the pixel spacing in the x, y, and z 
    dimensions, returning them as a list.

    Parameters:
    -----------
    tif_path : str
        Path to the TIFF file.

    Returns:
    --------
    original_spacing : list of float
        A list containing the pixel sizes in microns: [x_pixel_size_um, y_pixel_size_um, z_pixel_size_um].

    Notes:
    ------
    - The x and y resolutions are extracted from the TIFF XResolution and YResolution tags.
    - The z-spacing is inferred from the `IJMetadata` tag if available; otherwise, it falls back to 
      the `ImageDescription` tag.
    - Assumes the metadata is formatted in a way compatible with ImageJ or similar software.
    """
    with tifffile.TiffFile(tif_path) as tif:
        tif_tags = {}
        for tag in tif.pages[0].tags.values():
            name, value = tag.name, tag.value
            tif_tags[name] = value

        x_pixel_size_um = 1/((tif_tags["XResolution"])[0]/(tif_tags["XResolution"][1]))
        y_pixel_size_um = 1/((tif_tags["YResolution"])[0]/(tif_tags["YResolution"][1]))
        try:
            z_pixel_size_um = float(str(tif_tags["IJMetadata"]).split("nscales=")[1].split(",")[2].split("\\nunit")[0])
        except:
            z_pixel_size_um = (float(str(tif_tags["ImageDescription"]).split("spacing=")[1].split("loop")[0]))
       
    original_spacing = [x_pixel_size_um,y_pixel_size_um,z_pixel_size_um]
    return original_spacing

In [ ]:
def clean_labels(path, z_size = 6.0800000, x_y_size = 0.1700744):
    """
    Load and segment 3D or 4D label images, applying isotropic rescaling and nuclear seed-based watershed segmentation.

    Parameters:
    -----------
    path : str
        Path to the input image file (3D/4D grayscale label image) to be segmented. The image is assumed to have time as the first axis if 4D.
    z_size : float, optional
        The physical step size in the z-direction (µm) of the image stack. Default is 6.08 µm.
    x_y_size : float, optional
        The physical pixel size in the x and y directions (µm). Default is 0.1700744 µm.

    Returns:
    --------
    timelapse_labels : np.ndarray
        A 4D label array of shape (T, Z, Y, X), where each timepoint has been segmented using a seeded watershed approach.
        Labels below the estimated volume of a 2 µm radius nucleus are excluded.

    Method:
    -------
    1. Rescale the anisotropic stack to approach isotropic resolution in Z-Y-X (time unchanged).
    2. Apply Gaussian smoothing and Otsu thresholding to segment foreground regions.
    3. For each timepoint:
        a. Compute a distance transform on an eroded binary mask.
        b. Identify peak seed points for watershed segmentation using local maxima spaced ~4 µm apart.
        c. Perform marker-controlled watershed to segment candidate nuclei.
        d. Filter small regions below the estimated volume of a 2 µm radius sphere.
    4. Return the cleaned label array for all timepoints.

    Notes:
    ------
    - The physical voxel size is used to estimate a volume threshold for filtering small objects.
    - You can modify the `scale_value` to control isotropy tradeoffs (default is 0.5). Reduce if you have less compute power and vice versa
    - Output labels are returned as a NumPy array with time along the first axis.

    """
    image = convert(imread(path), 0,255, np.uint8)
    scale_change = z_size / x_y_size #how much bigger the z step is than x,y
    scale_value = 0.5
    new_pixel_size = (1/scale_value) * x_y_size

    #########CREATE LABELS
    rescaled_image = rescale(scale = (1,scale_value*scale_change, scale_value, scale_value), image=(image ), anti_aliasing = False)
    rescaled_image = gaussian(rescaled_image, sigma = 2)
    thresh = threshold_otsu(rescaled_image)
    rescaled_image = rescaled_image > thresh

    all_output_labels = []
    for t in range(rescaled_image.shape[0]):
        single_time = rescaled_image[t,:,:]
        distances = ndi.distance_transform_edt(erosion(single_time, ball(3)))
        coordinates = peak_local_max(distances, min_distance = int(20)) ####seed points at least 4um apart
        marker_locations = coordinates.data
        markers = np.zeros(single_time.shape, dtype=np.uint32)
        marker_indices = tuple(np.round(marker_locations).astype(int).T)
        markers[marker_indices] = np.arange(len(marker_locations)) + 1
        markers_big = dilation(markers, ball(2))
        segmented = watershed(-distances, markers_big, mask=single_time)
        table = regionprops_table(segmented, properties=('label', 'area'),)
        volume_threshold = (4/3)*np.pi * (4/new_pixel_size)**2     # radius 2um nucleus
        condition = (table['area'] >= volume_threshold)
        input_labels = table['label']
        output_labels = input_labels * condition
        output_labels = util.map_array(segmented, input_labels, output_labels)
        all_output_labels.append(output_labels)
    timelapse_labels = np.stack(all_output_labels, axis=0)
    return timelapse_labels

In [ ]:
def clean_images(path, z_size = 6.0800000, x_y_size = 0.1700744):
    """
    Load and rescale a DAPI fluorescence image stack (4D) to correct for anisotropy in Z-resolution.
    This allows you to view the images with labels overlaid in Napari viewer

    Parameters:
    -----------
    path : str
        Path to the input image file (assumed to be a 5D array: [T, C, Z, Y, X]).
        The function extracts the first channel (e.g. DAPI) from the image.

    z_size : float, optional
        The physical step size in the z-direction (µm) of the image stack. Default is 6.08 µm.

    x_y_size : float, optional
        The physical pixel size in the x and y directions (µm). Default is 0.1700744 µm.

    Returns:
    --------
    rescaled_image : np.ndarray
        A 4D array (T, Z, Y, X) where each frame has been rescaled to create isotropic voxel size.

    Method:
    -------
    1. Load the full image and extract only the first channel (assumed to be DAPI).
    2. Convert intensity range to 0–255 as `uint8`.
    3. Compute the anisotropy factor based on physical Z vs. X/Y spacing.
    4. Rescale Z/Y/X dimensions with a user-defined isotropy scaling factor (default 0.5).
       Time dimension remains unchanged.
    5. Return the rescaled image stack for viewing in napari.
    """
    all_times_dapi = imread(path)[:,:,0,:,:]
    image = convert(all_times_dapi, 0,255, np.uint8)
    print(image.shape)
    scale_change = scale_change = z_size / x_y_size #how much bigger the z step is than x,y
    scale_value = 0.5
    new_pixel_size = (1/scale_value) * x_y_size

    #########CREATE LABELS
    rescaled_image = rescale(scale = (1, scale_value*scale_change, scale_value, scale_value), image=(image ), anti_aliasing = False)
    return rescaled_image

In [27]:
binary_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\combined" # folder containing combined top+bottom binary segmentations
binary_paths = list(pathlib.Path(binary_root).rglob("*.{}".format("tif")))
combined_binary_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\stacked"
combined_label_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\unlinked_labels"
print(len(binary_paths))

44


## Stack Binary Labkit Segmentations

In [ ]:
unique_filenames = []
for path in binary_paths:
    filename = "_".join(os.path.basename(path).lower().split("_")[2:])
    if filename not in unique_filenames:
        unique_filenames.append(filename)
len(unique_filenames)

2

In [ ]:
all_images = []
for filename in unique_filenames:
    z_slices = []
    for path in binary_paths:
        if filename in str(path).lower():
            z_slices.append(path)
    z_slices.sort()
    all_images.append(z_slices)
files = all_images
sorted_files = natsort.natsorted(files)
    

In [20]:
sorted_images = []
for image in all_images:
    sorted_image = natsort.natsorted(image)
    sorted_images.append(sorted_image)

In [26]:
for image in sorted_images:
    frames = []
    for file in image:
        filename = "_".join(os.path.basename(file).lower().split("_")[2:]).split(".tif")[0]

        img = imread(file)


        frames.append(img)
    timelapse_labels = np.stack(frames, axis=0)
    print(timelapse_labels.shape)
    output_path = combined_binary_root+"\\{}_combined_timelapse.tif".format(filename)
    tifffile.imwrite(output_path, timelapse_labels, imagej=True, metadata={'axes': 'TZYX'})


(22, 11, 442, 456)
(22, 12, 409, 409)


## Convert Binary Segmentations into Unlinked Labels

In [31]:
combined_binary_images = list(pathlib.Path(combined_binary_root).rglob("*.{}".format("tif")))
for image in combined_binary_images:
    filename = "_".join(os.path.basename(image).lower().split("_")[2:]).replace(".tif", "")
    output_path = combined_label_root+"\\{}_FILTEREDLABELS.tif".format(filename)
    print(output_path)
    unlinked_labels = clean_labels(image)


    tifffile.imwrite(output_path, unlinked_labels.astype(np.float32), imagej=True, metadata={'axes': 'TZYX'})

C:\Users\itayl\Downloads\test_delete\out\unlinked_labels\24h-01-scene-04-p3_cropped_stiff_combined_timelapse_FILTEREDLABELS.tif
C:\Users\itayl\Downloads\test_delete\out\unlinked_labels\24h-01-scene-07-p9_cropped_stiff_combined_timelapse_FILTEREDLABELS.tif


In [ ]:
viewer = napari.Viewer()
viewer.add_labels(unlinked_labels)

<Labels layer 'labels' at 0x28fde1d3c10>

## Move to FIJI/ImageJ to Create Linked Labels
- If you have high temporal resolution you may be able to track in Python using TrackPy
- However, Trackmate allows you to match between frames based on object morphology as well as location